In [4]:
import json
import csv
import re


In [24]:
def writeMongo(row):
    print (row)
    
def conversion(old):
    #"40°29'21.84'' N"
    grados, minutos, segundos, direction = re.split(u'[°\'"\s]+',old)
    #print ('-'.join ([grados,minutos,segundos]))
    factor = {'N':1, 'S':-1, 'E': 1, 'W':-1}
    res= (int(grados)+int(minutos)/60.0+float(segundos)/3600.0) * factor[direction]
    return round(res,7)
latLon = conversion ("78°55'44.295'' N")
if 78.9289708 == latLon :
    print ("conversion ok")
else:
    print (latLon)

from unidecode import unidecode


def normalize(cadena):
    return unidecode(cadena.lower())

string_acentos = 'Ës un cáfÉ'

print (normalize(string_acentos))



conversion ok
es un cafe


In [29]:
jsonCallejero = open ('data/PermisosMadridTratado.csv','w')

i=0

with open ('data/PermisosMadrid.csv') as permisosIn:
        reader = csv.DictReader(permisosIn)
        for row in reader:
            rowOut = row
            for permiso in row['GRUPO PERMISOS']:
                rowOut = row
            name = row['VIA_CLASE']+' '+\
                   ("" if len(row['VIA_PAR'])==0 else row['VIA_PAR']+' ') +\
                   row['VIA_NOMBRE']
            name = normalize(name)
            if not 'name' in street:
                street['name']= name
                street['nums']= []
            elif street['name'] != name:
                json.dump(street,jsonCallejero)
                jsonCallejero.write('\n')
                i+=1
                street = {}
                street['name']=name
                street['nums']= []
            
            """ Miramos si 'num' ya existe en nums para los kilómetros de las carreteras,
                porque viene el del sentido de entrada y de salida, solo guardamos el de entrada
            """
            if (len(street['nums'])==0) or (row['NUMERO'] != street['nums'][-1]['num']) :
                num = {'num':row['NUMERO'], "location":{'type':'Point',
                                                        'coordinates':[ conversion(row['LATITUD']),
                                                                        conversion(row['LONGITUD'])]}}
                street['nums'].append(num)
        json.dump(street,jsonCallejero)
        jsonCallejero.write('\n')
        i+=1
        fCallejero.close()
        jsonCallejero.close()
        
        print ('total registros: {}'.format(i))
        

total registros: 8443


Con el fichero recultante de paso anterior lo importamos a Mongo en una instancia en Cloud de mongoDB.Atlas
```bash 
mongoimport --uri "mongodb+srv://sparkUser:<pass>@cluster0-emtpq.mongodb.net/test" --collection Callejero --drop --file Callejero.json
    
2019-04-24T16:21:39.496+0200    connected to: localhost
2019-04-24T16:21:39.535+0200    dropping: test.callejero
2019-04-24T16:21:42.154+0200    [########................] test.callejero       5.76MB/15.6MB (37.0%)
2019-04-24T16:21:45.155+0200    [####################....] test.callejero       13.3MB/15.6MB (85.3%)
2019-04-24T16:21:46.217+0200    [########################] test.callejero       15.6MB/15.6MB (100.0%)
2019-04-24T16:21:46.217+0200    imported 8443 documents
```

Posteriomente, mediante las utilidades de administración de la plataforma creamos un índice por el campo 'name' de tipo texto.

Los registros en la colección test.callejero tienen el siguiente formato:
```json
{
"_id":{"$oid":"5cc070f406ee493b07be37d2"},
"name":"calle de la abadesa",
"nums":[
   {"num":"1","location":{"type":"Point",
                          "coordinates":[{"$numberDouble":"40.4645222"},{"$numberDouble":"-3.7145222"}]}},
   {"num":"3","location":{"type":"Point",
                          "coordinates":[{"$numberDouble":"40.4645778"},{"$numberDouble":"-3.7145778"}]}},
   {"num":"5","location":{"type":"Point",
                          "coordinates":[{"$numberDouble":"40.4647167"},{"$numberDouble":"-3.7147167"}]}},
    ...
}
```

Hay que remarcar que el campo "location" es del tipo geoJson. Con el se harán cálculos de distancias para la geolocalización de las esquinas entre calles.